In [51]:
from pathlib import Path
import shutil
import numpy as np

In [52]:
RUN_DIR =    Path("/project2/andrewferguson/armin/FE_DATA/GHGGF_FEN/complex/usample_2d/prep/angles/runs")
MOLD_DIR =   Path("/project2/andrewferguson/armin/FE_DATA/GHGGF_FEN/complex/usample_2d/prep/angles/mold")
NUM_SUB = 4
NWINDOWS_ANG  = 18

# Preparations

In [53]:
def write_plumed(id, dist, ang, k_d, k_a, circ_id, plumed_f):
    plumed_cnt = [ "WHOLEMOLECULES ENTITY0=99-151 ENTITY1=1-98\n",
                   "# DEFINITIONS\n",
                   "MOLcom: COM ATOMS=99-151\n",
                   "PCCcom: COM ATOMS=1-98\n",
                   "# COLVARS\n",
                   "dcom: DISTANCE ATOMS=MOLcom,PCCcom\n",
                   "ang: ANGLE ATOMS=5,8,101,129\n",
                   "v1: DISTANCE ATOMS=5,8 SCALED_COMPONENTS NOPBC\n",
                   "v2: DISTANCE ATOMS=101,129 SCALED_COMPONENTS NOPBC\n",
                   "vr: DISTANCE ATOMS=PCCcom,2 SCALED_COMPONENTS NOPBC\n",
                   "crossx: CUSTOM ...\n",
                   "\tARG=v1.b,v1.c,v2.b,v2.c\n",
                   "\tVAR=y1,z1,y2,z2\n",
                   "\tFUNC=y1*z2-y2*z1\n",
                   "\tPERIODIC=NO\n",
                   "...\n",
                   "crossy: CUSTOM ...\n",
                   "\tARG=v1.a,v1.c,v2.a,v2.c\n",
                   "\tVAR=x1,z1,x2,z2\n",
                   "\tFUNC=x2*z1-x1*z2\n",
                   "\tPERIODIC=NO\n",
                   "...\n",
                   "crossz: CUSTOM ...\n",
                   "\tARG=v1.a,v1.b,v2.a,v2.b\n",
                   "\tVAR=x1,y1,x2,y2\n",
                   "\tFUNC=x1*y2-x2*y1\n",
                   "\tPERIODIC=NO\n",
                   "...\n",
                   "crosss: CUSTOM ...\n",
                   "\tARG=crossx,crossy,crossz\n",
                   "\tVAR=x,y,z\n",
                   "\tFUNC=sqrt(x*x+y*y+z*z)\n",
                   "\tPERIODIC=NO\n",
                   "...\n",
                   "refx: CUSTOM ...\n",
                   "\tARG=v1.b,v1.c,vr.b,vr.c\n",
                   "\tVAR=y1,z1,y2,z2\n",
                   "\tFUNC=y1*z2-y2*z1\n",
                   "\tPERIODIC=NO\n",
                   "...\n",
                   "refy: CUSTOM ...\n",
                   "\tARG=v1.a,v1.c,vr.a,vr.c\n",
                   "\tVAR=x1,z1,x2,z2\n",
                   "\tFUNC=x2*z1-x1*z2\n",
                   "\tPERIODIC=NO\n",
                   "...\n",
                   "refz: CUSTOM ...\n",
                   "\tARG=v1.a,v1.b,vr.a,vr.b\n",
                   "\tVAR=x1,y1,x2,y2\n",
                   "\tFUNC=x1*y2-x2*y1\n",
                   "\tPERIODIC=NO\n",
                   "...\n",
                   "refs: CUSTOM ...\n",
                   "\tARG=refx,refy,refz\n",
                   "\tVAR=x,y,z\n",
                   "\tFUNC=sqrt(x*x+y*y+z*z)\n",
                   "\tPERIODIC=NO\n",
                   "...\n",
                   "dc: CUSTOM ...\n",
                   "\tARG=crossx,crossy,crossz,refx,refy,refz,crosss,refs\n",
                   "\tVAR=x1,y1,z1,x2,y2,z2,A,B\n",
                   "\tFUNC=(x1*x2+y1*y2+z1*z2)/A/B\n",
                   "\tPERIODIC=NO\n",
                   "...\n",
                   "# BIAS\n",
                   f"udb: RESTRAINT ARG=dcom KAPPA={k_d} AT={dist}\n",
                   f"uab: RESTRAINT ARG=ang KAPPA={k_a} AT={ang}\n",
                   f"ucb: RESTRAINT ARG=dc KAPPA=150.0 AT={circ_id}\n",
                   "\n",
                   f"PRINT ARG=dcom,ang,dc,udb.bias,uab.bias,ucb.bias STRIDE=100 FILE=us_{id}.dat\n",
                    ]
    with open(plumed_f, 'w') as f:
        f.writelines(plumed_cnt)

In [54]:
ANGS = np.linspace(0, np.pi, NWINDOWS_ANG)

for i, ang in enumerate(ANGS):
    sim_dir = RUN_DIR/f"{i}"
    if sim_dir.exists():
        is_empty = not any(sim_dir.iterdir())
        if not is_empty:
            raise RuntimeError(f"Directory {i} already exists and is not empty.")
    else:
        sim_dir.mkdir()
    for file in MOLD_DIR.iterdir():
        shutil.copy(file, sim_dir/file.parts[-1])
    plumed_f = sim_dir/f"plumed_us.dat"
    write_plumed(i, 2.0, ang, 1000.00, 250.00, 1, plumed_f)

In [55]:
ANGS = ANGS[1:-1]
ANGS = ANGS[::-1]

for i, ang in enumerate(ANGS):
    ind = i + 18
    sim_dir = RUN_DIR/f"{ind}"
    if sim_dir.exists():
        is_empty = not any(sim_dir.iterdir())
        if not is_empty:
            raise RuntimeError(f"Directory {ind} already exists and is not empty.")
    else:
        sim_dir.mkdir()
    for file in MOLD_DIR.iterdir():
        shutil.copy(file, sim_dir/file.parts[-1])
    plumed_f = sim_dir/f"plumed_us.dat"
    write_plumed(ind, 2.0, ang, 1000.00, 250.00, -1, plumed_f)

# Submission

Just a little context manager for changing directories

In [56]:
import os
import contextlib

@contextlib.contextmanager
def working_directory(path):
    """Changes working directory and returns to previous on exit."""
    prev_cwd = Path.cwd()
    os.chdir(path)
    try:
        yield
    finally:
        os.chdir(prev_cwd)

In [57]:
import subprocess
import multiprocessing

def call_func(i):
   with working_directory(RUN_DIR/str(i)):
      print(f"{i}: ", end="")
      subprocess.call("sbatch -J GHGGF_us --wait sub_mdrun_plumed.sh", shell=True)
   return None

with multiprocessing.Pool(NUM_SUB) as pool:
   pool.map(call_func, range(2*NWINDOWS_ANG-2))

3: 0: 6: 9: Submitted batch job 31038922
Submitted batch job 31038923
Submitted batch job 31038924
Submitted batch job 31038925
4: 1: 7: 10: Submitted batch job 31038927
Submitted batch job 31038928
Submitted batch job 31038929
Submitted batch job 31038930
5: 2: 11: Submitted batch job 31038998
Submitted batch job 31038999
Submitted batch job 31039000
8: Submitted batch job 31039003
12: 15: 18: Submitted batch job 31039004
Submitted batch job 31039005
Submitted batch job 31039006
21: Submitted batch job 31039009
16: 13: 19: Submitted batch job 31039011
Submitted batch job 31039012
Submitted batch job 31039013
22: Submitted batch job 31039014
17: 14: 20: Submitted batch job 31039015
Submitted batch job 31039016
Submitted batch job 31039017
23: Submitted batch job 31039018
24: 27: 30: Submitted batch job 31039069
Submitted batch job 31039070
Submitted batch job 31039071
33: Submitted batch job 31039093
25: 28: Submitted batch job 31039095
Submitted batch job 31039096
31: Submitted batch 

In [58]:
import os
import asyncio
import telegram

async def notif(message):
    bot = telegram.Bot("")
    async with bot:
        await bot.send_message(chat_id=149407364, text=message)


await notif(f"{os.getcwd()} is done.")